# Cryptography HelloWorld

In [8]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Receiver e Emitter

A comunicação entre dois agentes com recurso à classe anteriormente definida é feita por meio de duas funções
`Emitter` e `Receiver` que enviam e recebem, respetivamente, uma mensagem de cada vez que são invocadas.
Ambos os intervenientes devem introduzir uma palavra passe que permitirá a posterior autenticação e cifragem
dos dados trocados entre os mesmos. Esta palavra passe é depois processada por um mecanismo de derivação de chaves
conhecido como *Password-Based Key Derivation Function* (**PBDKF**) que permite obter chaves de 
tamanho arbitrário a partir de dados de tamanho reduzido que combina com um valor aleatório conhecido como *salt*.

Por forma a garantir a confidenficialidade do conteúdo das mensagens bem como a autenticidade e integridade destas
e de dados associados (metadados) foi usada a cifra simétrica AES em modo GCM (*Gallois/Counter Mode*) que permite a 
autenticação de dados adicionais invocando o método:

```
cipher.authenticate_additional_data(header.encode("utf-8"))
```

Esta autenticação é feita via uma *tag* que deve ser introduzida no momento de decifragem dos dados e, caso não corresponda ao valor esperado, resulta numa exceção.

Para além da mensagem trocada entre os dois interveniente são ainda enviados dados que permitem a sua decifragem
por parte do `Receiver` nomeadamente o IV (*Initialization Vector*) usado na cifragem dos dados, a *tag* esperada
para decifrar os mesmos e ainda os metadados previamente autenticados:

```
payload = {'header': header, 'content': content, 'iv':iv, 'tag': cipher.tag}
        
conn.send(payload)
```

In [50]:
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

backend = default_backend()
cred_salt = os.urandom(16)

def Receiver(conn):
    pwd = getpass('Reciever password: ')
    try:
        kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=cred_salt, iterations=100000, backend=backend)
        key = kdf.derive(pwd.encode("utf-8"))
        payload = conn.recv()
        iv = payload['iv']
        tag = payload['tag']
        header = payload['header']
        ctxt = payload['content']

        decryptor = Cipher(algorithms.AES(key),modes.GCM(iv, tag),backend).decryptor()
        decryptor.authenticate_additional_data(header)
        print("Received: "+(decryptor.update(ctxt) + decryptor.finalize()).decode("utf-8"))
        conn.close()
    except:
        print("Receiver: Erro na autenticação da mensagem")

def Emitter(conn):
    pwd = getpass('Emitter password: ')
    try:
        kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=cred_salt, iterations=100000, backend=backend)
        key = kdf.derive(pwd.encode("utf-8"))
        iv = os.urandom(12)
        #AES in GCM mode to provide both confidentiality and authentication
        cipher = Cipher(algorithms.AES(key),modes.GCM(iv),backend).encryptor()

        header = 'msgheader'.encode("utf-8")
        content = 'Here is a sample message for testing'

        cipher.authenticate_additional_data(header)
        print("here")
        content = cipher.update(content.encode("utf-8")) + cipher.finalize()

        payload = {'header': header, 'content': content, 'iv':iv, 'tag': cipher.tag}

        conn.send(payload)
        conn.close()
    except:
       print("Emitter: Error")

In [49]:
Connection(Emitter, Receiver, timeout=10).manual()

Emitter password: ········
here
Reciever password: ········
Here is a sample message for testing
